# 数据分析简单代理

## 概述
本教程将指导您创建一个 AI 驱动的数据分析代理，该代理可以使用自然语言解释和回答有关数据集的问题。它将语言模型与数据操作工具相结合，实现直观的数据探索。

## 动机
数据分析通常需要专业知识，限制了非技术用户获取见解。通过创建一个理解自然语言查询的 AI 代理，我们可以使数据分析民主化，允许任何人从复杂的数据集中提取有价值的信息，而无需了解编程或统计工具。

## 关键组件
1. 语言模型：处理自然语言查询并生成类人响应
2. 数据操作框架：处理数据集操作和分析
3. 代理框架：将语言模型与数据操作工具连接起来
4. 合成数据集：出于演示目的代表真实世界的数据

## 方法
1. 创建一个代表汽车销售数据的合成数据集
2. 构建一个结合语言模型和数据分析能力的代理
3. 实现查询处理功能以处理自然语言问题
4. 通过示例查询演示代理的能力

## 结论
这种数据分析方法提供了显著的好处：
- 非技术用户的可访问性
- 处理各种查询类型的灵活性
- 高效的即席数据探索

通过使数据见解更容易获得，这种方法有可能改变组织在各个领域和行业中利用数据进行决策的方式。

## 导入库并设置环境变量

In [1]:
import os
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents import AgentType
from langchain_openai import ChatOpenAI
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Load environment variables
from dotenv import load_dotenv
import os

# Load environment variables and set OpenAI API key
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# Set a random seed for reproducibility
np.random.seed(42)

## 生成样本数据

在本节中，我们创建一个汽车销售的样本数据集。这包括生成日期、汽车品牌、型号、颜色和其他相关信息。

In [2]:
# Generate sample data
n_rows = 1000

# Generate dates
start_date = datetime(2022, 1, 1)
dates = [start_date + timedelta(days=i) for i in range(n_rows)]

# Define data categories
makes = ['Toyota', 'Honda', 'Ford', 'Chevrolet', 'Nissan', 'BMW', 'Mercedes', 'Audi', 'Hyundai', 'Kia']
models = ['Sedan', 'SUV', 'Truck', 'Hatchback', 'Coupe', 'Van']
colors = ['Red', 'Blue', 'Black', 'White', 'Silver', 'Gray', 'Green']

# Create the dataset
data = {
    'Date': dates,
    'Make': np.random.choice(makes, n_rows),
    'Model': np.random.choice(models, n_rows),
    'Color': np.random.choice(colors, n_rows),
    'Year': np.random.randint(2015, 2023, n_rows),
    'Price': np.random.uniform(20000, 80000, n_rows).round(2),
    'Mileage': np.random.uniform(0, 100000, n_rows).round(0),
    'EngineSize': np.random.choice([1.6, 2.0, 2.5, 3.0, 3.5, 4.0], n_rows),
    'FuelEfficiency': np.random.uniform(20, 40, n_rows).round(1),
    'SalesPerson': np.random.choice(['Alice', 'Bob', 'Charlie', 'David', 'Eva'], n_rows)
}

# Create DataFrame and sort by date
df = pd.DataFrame(data).sort_values('Date')

# Display sample data and statistics
print("\nFirst few rows of the generated data:")
print(df.head())

print("\nDataFrame info:")
df.info()

print("\nSummary statistics:")
print(df.describe())


First few rows of the generated data:
        Date       Make      Model  Color  Year     Price  Mileage  \
0 2022-01-01   Mercedes      Sedan  Green  2022  57952.65   5522.0   
1 2022-01-02  Chevrolet  Hatchback    Red  2021  58668.22  94238.0   
2 2022-01-03       Audi      Truck  White  2019  69187.87   7482.0   
3 2022-01-04     Nissan  Hatchback  Black  2016  40004.44  43846.0   
4 2022-01-05   Mercedes  Hatchback    Red  2016  63983.07  52988.0   

   EngineSize  FuelEfficiency SalesPerson  
0         2.0            24.7       Alice  
1         1.6            26.2         Bob  
2         2.0            28.0       David  
3         3.5            24.8       David  
4         2.5            24.1       Alice  

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            1000 non-null   datetime64[n

## 创建数据分析代理

在这里，我们使用 LangChain 创建一个 Pandas DataFrame 代理。该代理将能够分析我们的数据集并回答有关它的问题。

In [3]:
# Create the Pandas DataFrame agent
agent = create_pandas_dataframe_agent(
    ChatOpenAI(model="gpt-4o", temperature=0),
    df,
    verbose=True,
    allow_dangerous_code=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)
print("Data Analysis Agent is ready. You can now ask questions about the data.")

Data Analysis Agent is ready. You can now ask questions about the data.


## 定义提问函数

此函数允许我们轻松地向我们的数据分析代理提问并显示结果。

In [4]:
def ask_agent(question):
    """Function to ask questions to the agent and display the response"""
    response = agent.run({
        "input": question,
        "agent_scratchpad": f"Human: {question}\nAI: To answer this question, I need to use Python to analyze the dataframe. I'll use the python_repl_ast tool.\n\nAction: python_repl_ast\nAction Input: ",
    })
    print(f"Question: {question}")
    print(f"Answer: {response}")
    print("---")

## 示例问题

以下是您可以向数据分析代理提出的一些示例问题。您可以修改这些问题或添加您自己的问题来分析数据集。

In [5]:
# Example questions
ask_agent("What are the column names in this dataset?")
ask_agent("How many rows are in this dataset?")
ask_agent("What is the average price of cars sold?")

C:\Users\N7\AppData\Local\Temp\ipykernel_16872\610968568.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = agent.run({




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.columns.tolist()'}`


['Date', 'Make', 'Model', 'Color', 'Year', 'Price', 'Mileage', 'EngineSize', 'FuelEfficiency', 'SalesPerson']The column names in the dataset are:
1. Date
2. Make
3. Model
4. Color
5. Year
6. Price
7. Mileage
8. EngineSize
9. FuelEfficiency
10. SalesPerson

> Finished chain.
Question: What are the column names in this dataset?
Answer: The column names in the dataset are:
1. Date
2. Make
3. Model
4. Color
5. Year
6. Price
7. Mileage
8. EngineSize
9. FuelEfficiency
10. SalesPerson
---


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.shape[0]'}`


1000The dataset contains 1000 rows.

> Finished chain.
Question: How many rows are in this dataset?
Answer: The dataset contains 1000 rows.
---


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['Price'].mean()"}`


51145.360799999995The average price of cars s